In [1]:
from textblob import TextBlob # https://textblob.readthedocs.io/
from textblob import Blobber
from textblob.sentiments import NaiveBayesAnalyzer

from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import re
from IPython.display import clear_output

from pythonosc import osc_message_builder
from pythonosc import udp_client

In [3]:
tb = Blobber(analyzer=NaiveBayesAnalyzer())
driver = webdriver.Chrome()
driver.get("https://www.powerthesaurus.org/") # opens in new window
time.sleep(1)

try:
    while True:
        grabbed_text = (driver.find_element(By.XPATH, "/html/body").text)
        m = re.search('Just searched\n(.+?)\n', grabbed_text)
        if m:
            just_searched = m.group(1)

        blob = tb(just_searched)
        sentiment_positive = blob.sentiment.p_pos
        sentiment_negative = blob.sentiment.p_neg
        word_length = len(just_searched) + 0.0 # convert to float
        
        # send OSC message
        client = udp_client.UDPClient('127.0.0.1', 6448) # default wekinator address
        msg = osc_message_builder.OscMessageBuilder(address = '/wek/inputs')
        msg.add_arg(sentiment_positive)
        msg.add_arg(sentiment_negative)
        msg.add_arg(word_length)
        msg = msg.build()
        client.send(msg)

        print("Word : " + str(just_searched))
        print("Sentiment positive : " + str(sentiment_positive))
        print("Sentiment negative : " + str(sentiment_negative))
        print("Word length : " + str(word_length))
        time.sleep(1)
        clear_output(wait=True)                
              
except KeyboardInterrupt:
    driver.close() # close powerthesaurus window

Word : persiflage
Sentiment positive : 0.5
Sentiment negative : 0.5
Word length : 10.0
